In [0]:
!pip install tensorflow==2.0.0-beta0
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
from google.colab import files


In [0]:
uploaded = files.upload()
import io
df2 = pd.read_csv(io.BytesIO(uploaded['austin_weather.csv']))
X = df2[["Temp","Humidity","Pressure"]]
y = df2["Rain"]
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=122)
print(train_X.shape)
print(test_X.shape)
from tensorflow.keras import layers
model = tf.keras.Sequential()
model.add(layers.Dense(1, activation='relu', input_shape=(3,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='linear'))
model.compile(optimizer='rmsprop', loss='mse', metrics=['accuracy'])
model.summary()


In [0]:
history = model.fit(train_X, train_y, epochs=3000, batch_size=16, validation_data=(test_X, test_y))

In [0]:
model.predict([[[48,68,30]]])

In [0]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model to disk
open("sine_model.tflite", "wb").write(tflite_model)

# Convert the model to the TensorFlow Lite format with quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()

# Save the model to disk
open("sine_model_quantized.tflite", "wb").write(tflite_model)

In [0]:
# Instantiate an interpreter for each model
sine_model = tf.lite.Interpreter('sine_model.tflite')
sine_model_quantized = tf.lite.Interpreter('sine_model_quantized.tflite')

# Allocate memory for each model
sine_model.allocate_tensors()
sine_model_quantized.allocate_tensors()

# Get the input and output tensors so we can feed in values and get the results
sine_model_input = sine_model.tensor(sine_model.get_input_details()[0]["index"])
sine_model_output = sine_model.tensor(sine_model.get_output_details()[0]["index"])
sine_model_quantized_input = sine_model_quantized.tensor(sine_model_quantized.get_input_details()[0]["index"])
sine_model_quantized_output = sine_model_quantized.tensor(sine_model_quantized.get_output_details()[0]["index"])

# Create arrays to store the results
sine_model_predictions = np.empty(test_X.size)
sine_model_quantized_predictions = np.empty(test_X.size)

# Run each model's interpreter for each value and store the results in arrays

sine_model_input().put(0,100)
sine_model_input().put(1,20)
sine_model_input().put(2,50)
sine_model.invoke()
sine_model_predictions = sine_model_output()[0]
print(sine_model_predictions)
sine_model_quantized_input().put(0,100)
sine_model_quantized_input().put(1,20)
sine_model_quantized_input().put(2,50)
sine_model_quantized.invoke()
sine_model_quantized_predictions = sine_model_quantized_output()[0]


In [0]:
import os
basic_model_size = os.path.getsize("sine_model.tflite")
print("Basic model is %d bytes" % basic_model_size)
quantized_model_size = os.path.getsize("sine_model_quantized.tflite")
print("Quantized model is %d bytes" % quantized_model_size)
difference = basic_model_size - quantized_model_size
print("Difference is %d bytes" % difference)
# Install xxd if it is not available
!apt-get -qq install xxd
# Save the file as a C source file
!xxd -i sine_model_quantized.tflite > sine_model_quantized.cc
# Print the source file
!cat sine_model_quantized.cc